# Contents:

<!-- I. [Loading the data:](#Loading-the-data:)

II. [Model Building:](#Model-Building:)

* [Base Model:](#Base-Model:)

* [Automatic Hyper parameter tuning:](#Automatic-Hyper-parameter-tuning:)
    * [Model Data Monitoring:](#Model-Data-Monitoring:)
    * [Deploying the model with Data Capture:](#Deploying-the-model-with-Data-Capture:)
    * [Invoking the endpoint using Boto3: (way-1)](#Invoking-the-endpoint-using-Boto3:-(way-1))
    * [Invoking the Endpoint: (way-2)](#Invoking-the-Endpoint:-(way-2))
    * [Explore the captured data:](#Explore-the-captured-data:)

III. [Model Data Monitor - Baselining and continuous monitoring:](#Model-Data-Monitor-:-Baselining-and-continuous-monitoring:)

* [1. Constraint suggestion with baseline/training dataset](#1.-Constraint-suggestion-with-baseline/training-dataset)

* [2. Creating a Schedule and Analyzing collected data for data quality issues](#2.-Creating-a-Schedule-and-Analyzing-collected-data-for-data-quality-issues)
    * [Create a schedule](#Create-a-schedule)
    * [Generating some artificial traffic](#Generating-some-artificial-traffic)
    * [Describe and inspect the schedule](#Describe-and-inspect-the-schedule)
    * [Analyze the Executions](#Analyze-the-Executions) -->

I. [Loading the data:](#Loading-the-data:)

II. [Model Building:](#Model-Building:)

* [Base Model:](#Base-Model:)

* [Automatic Hyper parameter tuning:](#Automatic-Hyper-parameter-tuning:)

III. [Model Data Monitor : Capturing, Baselining and Continuous monitoring:](#Model-Data-Monitor-:-Capturing,-Baselining-and-Continuous-monitoring:)

* [1. Data Capturing:](#1.-Data-Capturing:)
    * [Deploying the model with Data Capture:](#Deploying-the-model-with-Data-Capture:)
    * [Invoking the endpoint using Boto3: (way-1)](#Invoking-the-endpoint-using-Boto3:-(way-1))
    * [Invoking the Endpoint: (way-2)](#Invoking-the-Endpoint:-(way-2))
    * [Explore the captured data:](#Explore-the-captured-data:)


* [2. Constraint suggestion with baseline/training dataset](#2.-Constraint-suggestion-with-baseline/training-dataset)

* [3. Creating a Schedule and Analyzing collected data for data quality issues](#3.-Creating-a-Schedule-and-Analyzing-collected-data-for-data-quality-issues)
    * [Create a schedule](#Create-a-schedule)
    * [Generating some artificial traffic](#Generating-some-artificial-traffic)
    * [Describe and inspect the schedule](#Describe-and-inspect-the-schedule)
    * [Analyze the Executions](#Analyze-the-Executions)

# **Loading the data:**

([Contents:](#Contents:))

Conditions for data while modeling in AWS:

* Target variable should be the first column
* The data should not have any headers

In [20]:
# Importing the necessary libraries
import pandas as pd 
import numpy as np 
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", 200)

import matplotlib.pyplot as plt
import seaborn as sns

import io
import boto3
import sagemaker
from sagemaker.predictor import csv_serializer,csv_deserializer
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session
from sagemaker import get_execution_role

import warnings
warnings.filterwarnings('ignore')

In [39]:
# Specify the resource names and roles
role = get_execution_role()
bucket = 'accidentbucket'
# prefix = 'Data-Processing-2022-08-02T17-07-37' 
prefix = 'Data-Processing-Trial3-2022-08-24T23-47-43' # s3://accidentbucket/Data-Processing-Trial3-2022-08-24T23-47-43/
my_region = boto3.session.Session().region_name

print("AWS Bucket: ", bucket)
print("Prefix (or Subdirectory): ", prefix)
print("AWS Region: ", my_region)

AWS Bucket:  accidentbucket
Prefix (or Subdirectory):  Data-Processing-Trial3-2022-08-24T23-47-43
AWS Region:  us-east-1


In [40]:
# Using Boto3 to create a connection
s3_client = boto3.client('s3')
contents = s3_client.list_objects(Bucket=bucket, Prefix=prefix)['Contents']
for f in contents:
    print(f['Key'])

Data-Processing-Trial3-2022-08-24T23-47-43/part-00000-95dae8bf-d75d-4c62-9dc1-4fe7805b684c-c000.csv


In [41]:
s3_client.list_objects(Bucket=bucket, Prefix=prefix)

{'ResponseMetadata': {'RequestId': 'PC2B8B04KQGWRFY6',
  'HostId': '/2qiQjpr+iDoGa/HR2XK0XQOLBiHqjZyzHeb3kYS65dhxnVM356Lvtybvh+flh9wCiEVo8Grz9M=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/2qiQjpr+iDoGa/HR2XK0XQOLBiHqjZyzHeb3kYS65dhxnVM356Lvtybvh+flh9wCiEVo8Grz9M=',
   'x-amz-request-id': 'PC2B8B04KQGWRFY6',
   'date': 'Thu, 25 Aug 2022 02:09:32 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'Data-Processing-Trial3-2022-08-24T23-47-43/part-00000-95dae8bf-d75d-4c62-9dc1-4fe7805b684c-c000.csv',
   'LastModified': datetime.datetime(2022, 8, 24, 18, 24, 57, tzinfo=tzlocal()),
   'ETag': '"81844ae851e5eec92edf317d1d0175e1"',
   'Size': 7892940,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'aws',
    'ID': 'd4a4229a6a4db7cc4f57f9bad2f6023ff80c93d342fb330a153cdf48899ef836'}}],
 'Name':

In [42]:
contents[0]['Key']

'Data-Processing-Trial3-2022-08-24T23-47-43/part-00000-95dae8bf-d75d-4c62-9dc1-4fe7805b684c-c000.csv'

In [43]:
# Get the S3 uri location for the processed dataset (Option-1)
file_obj = s3_client.get_object(Bucket=bucket,Key=contents[0]['Key'])
df = pd.read_csv(io.BytesIO(file_obj['Body'].read()))
print(df.shape)
df.head()

(47406, 38)


,reported_reason_of_death,target_encoded,age_years,insurance_grp,no_fam_member,total_visit_count,diagnosis_code_1,diagnosis_code_2,diagnosis_code_3,p1,p2,p3,alcohol,drug,fire,sex_M,sex_F,country_of_origin_V,country_of_origin_C,country_of_origin_L,country_of_origin_S,country_of_origin_M,place_of_death_2,place_of_death_0,place_of_death_1,place_of_incident_Home,place_of_incident_Not recorded,place_of_incident_School/ Sports Complex,place_of_incident_Public property,place_of_incident_Street,place_of_incident_Unknown,place_of_incident_Industrial area,age_bins_less than 20,age_bins_60 above,age_bins_20 to 40,age_bins_41 to 60,insurance_grp_bin_less than 50,insurance_grp_bin_greater than 50
0,Heart attack,0,71,46.0,1.0,1.0,62.0,-1.0,-1.0,1893.0,1820.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,Hemorrhagic stroke,1,0,46.0,1.0,0.0,53.0,-1.0,-1.0,4010.0,1807.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,Heart attack,0,0,31.0,1.0,1.0,53.0,-1.0,-1.0,4076.0,1807.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,Heart attack,0,5,31.0,1.0,1.0,59.0,75.0,62.0,1807.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,Heart attack,0,3,31.0,1.0,2.0,59.0,-1.0,-1.0,4076.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [44]:
# file_obj

In [45]:
# Directly using s3 uri (Option-2)
s3_location = 's3://{}/{}'.format(bucket,contents[0]['Key'])
print(s3_location)
df = pd.read_csv(s3_location)
print(df.shape)
df.head()

s3://accidentbucket/Data-Processing-Trial3-2022-08-24T23-47-43/part-00000-95dae8bf-d75d-4c62-9dc1-4fe7805b684c-c000.csv
(47406, 38)


,reported_reason_of_death,target_encoded,age_years,insurance_grp,no_fam_member,total_visit_count,diagnosis_code_1,diagnosis_code_2,diagnosis_code_3,p1,p2,p3,alcohol,drug,fire,sex_M,sex_F,country_of_origin_V,country_of_origin_C,country_of_origin_L,country_of_origin_S,country_of_origin_M,place_of_death_2,place_of_death_0,place_of_death_1,place_of_incident_Home,place_of_incident_Not recorded,place_of_incident_School/ Sports Complex,place_of_incident_Public property,place_of_incident_Street,place_of_incident_Unknown,place_of_incident_Industrial area,age_bins_less than 20,age_bins_60 above,age_bins_20 to 40,age_bins_41 to 60,insurance_grp_bin_less than 50,insurance_grp_bin_greater than 50
0,Heart attack,0,71,46.0,1.0,1.0,62.0,-1.0,-1.0,1893.0,1820.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,Hemorrhagic stroke,1,0,46.0,1.0,0.0,53.0,-1.0,-1.0,4010.0,1807.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,Heart attack,0,0,31.0,1.0,1.0,53.0,-1.0,-1.0,4076.0,1807.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,Heart attack,0,5,31.0,1.0,1.0,59.0,75.0,62.0,1807.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,Heart attack,0,3,31.0,1.0,2.0,59.0,-1.0,-1.0,4076.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [46]:
# Random split
train_data, test_data = np.split(df.sample(frac=1, random_state=123), [int(0.70 * len(df))])
print(train_data.shape, test_data.shape)

(33184, 38) (14222, 38)


In [47]:
train_data.head(3)

,reported_reason_of_death,target_encoded,age_years,insurance_grp,no_fam_member,total_visit_count,diagnosis_code_1,diagnosis_code_2,diagnosis_code_3,p1,p2,p3,alcohol,drug,fire,sex_M,sex_F,country_of_origin_V,country_of_origin_C,country_of_origin_L,country_of_origin_S,country_of_origin_M,place_of_death_2,place_of_death_0,place_of_death_1,place_of_incident_Home,place_of_incident_Not recorded,place_of_incident_School/ Sports Complex,place_of_incident_Public property,place_of_incident_Street,place_of_incident_Unknown,place_of_incident_Industrial area,age_bins_less than 20,age_bins_60 above,age_bins_20 to 40,age_bins_41 to 60,insurance_grp_bin_less than 50,insurance_grp_bin_greater than 50
14593,Hemorrhagic stroke,1,5,37.0,1.0,0.0,59.0,-1.0,-1.0,4076.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
34710,Heart attack,0,24,2.0,1.0,0.0,62.0,-1.0,-1.0,1864.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
16543,Heart attack,0,14,32.0,1.0,1.0,52.0,-1.0,-1.0,661.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [48]:
test_data.head(3)

,reported_reason_of_death,target_encoded,age_years,insurance_grp,no_fam_member,total_visit_count,diagnosis_code_1,diagnosis_code_2,diagnosis_code_3,p1,p2,p3,alcohol,drug,fire,sex_M,sex_F,country_of_origin_V,country_of_origin_C,country_of_origin_L,country_of_origin_S,country_of_origin_M,place_of_death_2,place_of_death_0,place_of_death_1,place_of_incident_Home,place_of_incident_Not recorded,place_of_incident_School/ Sports Complex,place_of_incident_Public property,place_of_incident_Street,place_of_incident_Unknown,place_of_incident_Industrial area,age_bins_less than 20,age_bins_60 above,age_bins_20 to 40,age_bins_41 to 60,insurance_grp_bin_less than 50,insurance_grp_bin_greater than 50
24640,Heart attack,0,15,47.0,1.0,1.0,52.0,-1.0,-1.0,3287.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
35381,Heart attack,0,88,50.0,1.0,2.0,62.0,-1.0,-1.0,1807.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
33726,Heart attack,0,0,10.0,1.0,4.0,59.0,76.0,58.0,1519.0,1679.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [49]:
# Getting the label map dictionary
temp = df.groupby(['reported_reason_of_death','target_encoded']).count().reset_index()
display(temp.head(5))
label_map = dict(zip(temp['reported_reason_of_death'].values, temp['target_encoded'].values))
print(label_map)

# Saving it as a pickle file for s3 upload
import pickle
label_file = 'target_label_map.pkl'
with open(label_file,'wb') as f:
    pickle.dump(label_map,f)
    
# Uploading the file to s3 bucket for future use
s3_client.upload_file(Filename=label_file,Bucket=bucket,Key='target_label_map')

,reported_reason_of_death,target_encoded,age_years,insurance_grp,no_fam_member,total_visit_count,diagnosis_code_1,diagnosis_code_2,diagnosis_code_3,p1,p2,p3,alcohol,drug,fire,sex_M,sex_F,country_of_origin_V,country_of_origin_C,country_of_origin_L,country_of_origin_S,country_of_origin_M,place_of_death_2,place_of_death_0,place_of_death_1,place_of_incident_Home,place_of_incident_Not recorded,place_of_incident_School/ Sports Complex,place_of_incident_Public property,place_of_incident_Street,place_of_incident_Unknown,place_of_incident_Industrial area,age_bins_less than 20,age_bins_60 above,age_bins_20 to 40,age_bins_41 to 60,insurance_grp_bin_less than 50,insurance_grp_bin_greater than 50
0,Heart attack,0,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000,28000
1,Hemorrhagic stroke,1,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902,14902
2,Other,3,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009
3,Suicide,2,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495,2495


{'Heart attack': 0, 'Hemorrhagic stroke': 1, 'Other': 3, 'Suicide': 2}


In [50]:
# # How to load the pickle file from s3 for future use
# s3_client.download_file(Bucket=bucket,Key='target_label_map',Filename='target_label_map')

# with open('target_label_map','rb') as F:
#     map_ = pickle.load(F)

In [51]:
# Dropping the label column (we already have encoded version of the label for training purpose)
train_data = train_data.drop('reported_reason_of_death',axis=1)
test_data = test_data.drop('reported_reason_of_death',axis=1)

In [52]:
# Uploading the training data to s3
pd.concat([train_data['target_encoded'], train_data.drop(['target_encoded'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
s3_client.upload_file(Filename='train.csv',Bucket=bucket,Key='data/train/train.csv')

# Creating the Training input for modeling
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train.csv'.format(bucket, 'data/train'), content_type='csv')

In [53]:
# Uploading the test data to s3
pd.concat([test_data['target_encoded'], test_data.drop(['target_encoded'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
s3_client.upload_file(Filename='test.csv',Bucket=bucket,Key='data/test/test.csv')

# Creating the Test input for modeling
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test.csv'.format(bucket, 'data/test'), content_type='csv')

In [54]:
# set an output path where the trained model will be saved
subdir = 'model/training'
output_path ='s3://{}/{}/output'.format(bucket, subdir)
print(output_path)

s3://accidentbucket/model/training/output


# **Model Building:**

([Contents:](#Contents:))

## **Base Model:**

In [55]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = sagemaker.image_uris.retrieve(region=my_region,
                                         framework='xgboost',
                                         version='latest')
print(container)

811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest


In [56]:
# create the sagemaker session
sess = sagemaker.Session()

# Instantiate the model object
xgb = sagemaker.estimator.Estimator(image_uri = container,
                                   role = role,
                                   instance_count=1, 
                                   instance_type='ml.m5.xlarge',
                                   volume_size=4,
                                    output_path=output_path,
                                    sagemaker_session=sess
                                   )

# Set the hyperparameters
xgb.set_hyperparameters(objective= 'multi:softmax',num_class=4,num_round=500,max_depth=5)
print(xgb)

In [57]:
# Fit the model
xgb.fit({'train': s3_input_train,'validation': s3_input_test},wait=True)

2022-08-25 02:29:27 Starting - Starting the training job...
2022-08-25 02:29:51 Starting - Preparing the instances for trainingProfilerReport-1661394567: InProgress
......
2022-08-25 02:30:54 Downloading - Downloading input data...
2022-08-25 02:31:14 Training - Downloading the training image.....Arguments: train
[2022-08-25:02:32:03:INFO] Running standalone xgboost training.
[2022-08-25:02:32:03:INFO] File size need to be processed in the node: 6.88mb. Available memory size in the node: 8324.2mb
[2022-08-25:02:32:03:INFO] Determined delimiter of CSV input is ','
[02:32:03] S3DistributionType set as FullyReplicated
[02:32:03] 33184x36 matrix with 1194624 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-08-25:02:32:03:INFO] Determined delimiter of CSV input is ','
[02:32:03] S3DistributionType set as FullyReplicated
[02:32:03] 14222x36 matrix with 511992 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[02:32

In [58]:
# Deploying the xgb model
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer

xgb_predictor = xgb.deploy(endpoint_name='xgb-test-2',
	initial_instance_count = 1,
	instance_type = 'ml.m4.xlarge',
	serializer = CSVSerializer())

--------!

In [59]:
xgb_predictor.endpoint_name

'xgb-test-2'

In [61]:
# Evaluating the model on the test data
test_data_array = test_data.drop(['target_encoded'], axis=1).values #load the data into an array
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
print(predictions[:20])
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)
print(predictions_array[:10])

0.0,0.0,0.0,0.0,1.0,
(14222,)
[0. 0. 0. 0. 1. 0. 0. 0. 2. 1.]


In [62]:
test_data.shape

(14222, 37)

In [63]:
from sklearn import metrics

print("Test Data:--------------------------------------")
accuracy = metrics.accuracy_score(test_data.target_encoded.values,predictions_array)
print(f"Accuracy: {accuracy}\n")
print(metrics.classification_report(test_data.target_encoded.values,predictions_array))

Test Data:--------------------------------------
Accuracy: 0.8471382365349459

              precision    recall  f1-score   support

           0       0.91      0.85      0.88      8412
           1       0.74      0.84      0.79      4489
           2       0.77      0.70      0.73       742
           3       1.00      1.00      1.00       579

    accuracy                           0.85     14222
   macro avg       0.86      0.85      0.85     14222
weighted avg       0.85      0.85      0.85     14222



### Invoking the endpoint using Boto3:

In [64]:
# getting the runtime object
runtime = boto3.client("sagemaker-runtime")
endpoint_name = 'xgb-test-2'
endpoint_name

'xgb-test-2'

In [65]:
test_data.head()

,target_encoded,age_years,insurance_grp,no_fam_member,total_visit_count,diagnosis_code_1,diagnosis_code_2,diagnosis_code_3,p1,p2,p3,alcohol,drug,fire,sex_M,sex_F,country_of_origin_V,country_of_origin_C,country_of_origin_L,country_of_origin_S,country_of_origin_M,place_of_death_2,place_of_death_0,place_of_death_1,place_of_incident_Home,place_of_incident_Not recorded,place_of_incident_School/ Sports Complex,place_of_incident_Public property,place_of_incident_Street,place_of_incident_Unknown,place_of_incident_Industrial area,age_bins_less than 20,age_bins_60 above,age_bins_20 to 40,age_bins_41 to 60,insurance_grp_bin_less than 50,insurance_grp_bin_greater than 50
24640,0,15,47.0,1.0,1.0,52.0,-1.0,-1.0,3287.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
35381,0,88,50.0,1.0,2.0,62.0,-1.0,-1.0,1807.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
33726,0,0,10.0,1.0,4.0,59.0,76.0,58.0,1519.0,1679.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7096,0,13,32.0,1.0,2.0,62.0,-1.0,-1.0,1211.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
34102,1,34,47.0,1.0,1.0,59.0,82.0,59.0,4014.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [66]:
# converting the data to bytes array format to make it work with invoke_endpoint method
csv_object = test_data.iloc[:,1:].to_csv(header=False, index=False).encode("utf-8")
csv_object[:200]

b'15,47.0,1.0,1.0,52.0,-1.0,-1.0,3287.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0\n88,50.0,1.0,2.0,62.0,-1.0,-1.0,1807.0,0.0,0.0,0.0,'

In [72]:
# csv serialization
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=csv_object,
    ContentType="text/csv",
)

output_ = np.array(response["Body"].read().decode('utf-8'))
print(type(output_))

<class 'numpy.ndarray'>


In [70]:
print(response)

{'ResponseMetadata': {'RequestId': '9d15b489-711d-4456-83d9-59cbf2d88983', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9d15b489-711d-4456-83d9-59cbf2d88983', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Thu, 25 Aug 2022 03:00:45 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '56887'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f49a75b4910>}


In [73]:
output_ = output_.reshape(-1)[0].split(',')
output_ = [float(i) for i in output_]
output_[:5]

[0.0, 0.0, 0.0, 0.0, 1.0]

In [74]:
actual_output = test_data['target_encoded'].values

from sklearn import metrics

print("Test Data:--------------------------------------")
accuracy = metrics.accuracy_score(actual_output,output_)
print(f"Accuracy: {accuracy}\n")
print(metrics.classification_report(actual_output,output_))

Test Data:--------------------------------------
Accuracy: 0.8471382365349459

              precision    recall  f1-score   support

           0       0.91      0.85      0.88      8412
           1       0.74      0.84      0.79      4489
           2       0.77      0.70      0.73       742
           3       1.00      1.00      1.00       579

    accuracy                           0.85     14222
   macro avg       0.86      0.85      0.85     14222
weighted avg       0.85      0.85      0.85     14222



## **Automatic Hyper parameter tuning:**

([Contents:](#Contents:))

Amazon SageMaker automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose. For example, suppose that you want to solve a binary classification problem on this marketing dataset. Your goal is to maximize the area under the curve (auc) metric of the algorithm by training an XGBoost Algorithm model. You don't know which values of the eta, alpha, min_child_weight, and max_depth hyperparameters to use to train the best model. To find the best values for these hyperparameters, you can specify ranges of values that Amazon SageMaker hyperparameter tuning searches to find the combination of values that results in the training job that performs the best as measured by the objective metric that you chose. Hyperparameter tuning launches training jobs that use hyperparameter values in the ranges that you specified, and returns the training job with highest auc.

1. We will tune four hyperparameters in this example:
* **eta:** Step size shrinkage used in updates to prevent overfitting. After each boosting step, you can directly get the weights of new features. The eta parameter actually shrinks the feature weights to make the boosting process more conservative.
* **min_child_weight:** Minimum sum of instance weight (hessian) needed in a child. If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, the building process gives up further partitioning. In linear regression models, this simply corresponds to a minimum number of instances needed in each node. The larger the algorithm, the more conservative it is.
* **alpha:** L1 regularization term on weights. Increasing this value makes models more conservative.
* **max_depth:** Maximum depth of a tree. Increasing this value makes the model more complex and likely to be overfitted.

In [75]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, CategoricalParameter, HyperparameterTuner

hyperparameter_range = {'eta': ContinuousParameter(0,1),
                        'min_child_weight': ContinuousParameter(1,30),
                         'alpha': ContinuousParameter(0,10),
                         'max_depth': IntegerParameter(1,10)
                       }

2. Next we'll specify the objective metric that we'd like to tune and its definition, which includes the regular expression (Regex) needed to extract that metric from the CloudWatch logs of the training job. Since we are using built-in XGBoost algorithm here, it emits two predefined metrics: validation:merror and train:merror, and we elected to monitor validation:merror as you can see below. In this case, we only need to specify the metric name and do not need to provide regex. If you bring your own algorithm, your algorithm emits metrics by itself. In that case, you'll need to add a MetricDefinition object here to define the format of those metrics through regex, so that SageMaker knows how to extract those metrics from your CloudWatch logs.

In [76]:
objective_metric_name = 'validation:merror'

3. Now, we'll create a HyperparameterTuner object, to which we pass:
* The XGBoost estimator we created above
* Our hyperparameter ranges
* Objective metric name and definition
* Tuning resource configurations such as Number of training jobs to run in total and how many training jobs can be run in parallel.


In [77]:
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_range,
                            max_jobs=12,
                            max_parallel_jobs=3,
                            base_tuning_job_name = 'xgbtest2',
                            objective_type = 'Minimize'
                           )

In [78]:
tuner

4. Now we can launch a hyperparameter tuning job by calling fit() function. After the hyperparameter tuning job is created, we can go to SageMaker console to track the progress of the hyperparameter tuning job until it is completed.

In [79]:
# Fit the tuning job
tuner.fit({'train': s3_input_train,'validation': s3_input_test},wait=True)

.........................................................................................................................!


In [80]:
# Tuner summary
tuner.describe()

{'HyperParameterTuningJobName': 'xgbtest2-220825-0315',
 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:143176219551:hyper-parameter-tuning-job/xgbtest2-220825-0315',
 'HyperParameterTuningJobConfig': {'Strategy': 'Bayesian',
  'HyperParameterTuningJobObjective': {'Type': 'Minimize',
   'MetricName': 'validation:merror'},
  'ResourceLimits': {'MaxNumberOfTrainingJobs': 12,
   'MaxParallelTrainingJobs': 3},
  'ParameterRanges': {'IntegerParameterRanges': [{'Name': 'max_depth',
     'MinValue': '1',
     'MaxValue': '10',
     'ScalingType': 'Auto'}],
   'ContinuousParameterRanges': [{'Name': 'eta',
     'MinValue': '0',
     'MaxValue': '1',
     'ScalingType': 'Auto'},
    {'Name': 'min_child_weight',
     'MinValue': '1',
     'MaxValue': '30',
     'ScalingType': 'Auto'},
    {'Name': 'alpha',
     'MinValue': '0',
     'MaxValue': '10',
     'ScalingType': 'Auto'}],
   'CategoricalParameterRanges': []},
  'TrainingJobEarlyStoppingType': 'Off'},
 'TrainingJobDefinition': 

In [81]:
# Best Training job name
tuner.best_training_job()

'xgbtest2-220825-0315-006-10eba8fa'

# **Model Data Monitor : Capturing, Baselining and Continuous monitoring:**

([Contents:](#Contents:))

There are 3 steps to do the Model & Data monitoring in AWS:

1. Capturing the data that will be sent to the deployed endpoint (Data Capturing)
2. Creating a baseline for the data on which the model is trained (Baselining)
3. Creating a monitoring schedule which compares the captured data with the baseline data statistics for any deviations

### **1. Data Capturing:**

### **Deploying the model with Data Capture:**

([Contents:](#Contents:))

In [82]:
# Data capture config
from sagemaker.model_monitor import DataCaptureConfig

# Path where we need to store the data that will be captured after the model is deployed
data_capture_prefix = 'model_monitor/datacapture'
s3_capture_upload_path = 's3://{}/{}'.format(bucket, data_capture_prefix)
print(s3_capture_upload_path)

endpoint_name = 'tuned-xgboost-3'
print("EndpointName={}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
                        enable_capture=True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path)

s3://accidentbucket/model_monitor/datacapture
EndpointName=tuned-xgboost-3


In [83]:
# Deploy the best trained model
tuner_predictor = tuner.deploy(initial_instance_count=1,
                               endpoint_name=endpoint_name,
                               instance_type = 'ml.m4.xlarge',
                               serializer = CSVSerializer(),
                               data_capture_config=data_capture_config
                              )


2022-08-25 03:21:02 Starting - Found matching resource for reuse
2022-08-25 03:21:02 Downloading - Downloading input data
2022-08-25 03:21:02 Training - Training image download completed. Training in progress.
2022-08-25 03:21:02 Uploading - Uploading generated training model
2022-08-25 03:21:02 Completed - Resource reused by training job: xgbtest2-220825-0315-007-051ca6c0
---------------!

In [84]:
tuner_predictor.endpoint_name

'tuned-xgboost-3'

In [85]:
# Evaluating the model on the test data

test_data_array = test_data.drop(['target_encoded'], axis=1).values #load the data into an array
predictions = tuner_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(14222,)


In [86]:
from sklearn import metrics

print("Test Data:--------------------------------------")
accuracy = metrics.accuracy_score(test_data.target_encoded.values,predictions_array)
print(f"Accuracy: {accuracy}\n")
print(metrics.classification_report(test_data.target_encoded.values,predictions_array))

Test Data:--------------------------------------
Accuracy: 0.8550133595837435

              precision    recall  f1-score   support

           0       0.93      0.84      0.89      8412
           1       0.74      0.88      0.80      4489
           2       0.76      0.73      0.74       742
           3       1.00      1.00      1.00       579

    accuracy                           0.86     14222
   macro avg       0.86      0.86      0.86     14222
weighted avg       0.87      0.86      0.86     14222



In [87]:
test_data.head()

,target_encoded,age_years,insurance_grp,no_fam_member,total_visit_count,diagnosis_code_1,diagnosis_code_2,diagnosis_code_3,p1,p2,p3,alcohol,drug,fire,sex_M,sex_F,country_of_origin_V,country_of_origin_C,country_of_origin_L,country_of_origin_S,country_of_origin_M,place_of_death_2,place_of_death_0,place_of_death_1,place_of_incident_Home,place_of_incident_Not recorded,place_of_incident_School/ Sports Complex,place_of_incident_Public property,place_of_incident_Street,place_of_incident_Unknown,place_of_incident_Industrial area,age_bins_less than 20,age_bins_60 above,age_bins_20 to 40,age_bins_41 to 60,insurance_grp_bin_less than 50,insurance_grp_bin_greater than 50
24640,0,15,47.0,1.0,1.0,52.0,-1.0,-1.0,3287.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
35381,0,88,50.0,1.0,2.0,62.0,-1.0,-1.0,1807.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
33726,0,0,10.0,1.0,4.0,59.0,76.0,58.0,1519.0,1679.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7096,0,13,32.0,1.0,2.0,62.0,-1.0,-1.0,1211.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
34102,1,34,47.0,1.0,1.0,59.0,82.0,59.0,4014.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


### **Invoking the endpoint using Boto3: (way-1)**

([Contents:](#Contents:))

In [101]:
# getting the runtime object
runtime = boto3.client("sagemaker-runtime")
endpoint_name = 'tuned-xgboost-3'
endpoint_name

'tuned-xgboost-3'

In [102]:
test_data.head()

,target_encoded,age_years,insurance_grp,no_fam_member,total_visit_count,diagnosis_code_1,diagnosis_code_2,diagnosis_code_3,p1,p2,p3,alcohol,drug,fire,sex_M,sex_F,country_of_origin_V,country_of_origin_C,country_of_origin_L,country_of_origin_S,country_of_origin_M,place_of_death_2,place_of_death_0,place_of_death_1,place_of_incident_Home,place_of_incident_Not recorded,place_of_incident_School/ Sports Complex,place_of_incident_Public property,place_of_incident_Street,place_of_incident_Unknown,place_of_incident_Industrial area,age_bins_less than 20,age_bins_60 above,age_bins_20 to 40,age_bins_41 to 60,insurance_grp_bin_less than 50,insurance_grp_bin_greater than 50
24640,0,15,47.0,1.0,1.0,52.0,-1.0,-1.0,3287.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
35381,0,88,50.0,1.0,2.0,62.0,-1.0,-1.0,1807.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
33726,0,0,10.0,1.0,4.0,59.0,76.0,58.0,1519.0,1679.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7096,0,13,32.0,1.0,2.0,62.0,-1.0,-1.0,1211.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
34102,1,34,47.0,1.0,1.0,59.0,82.0,59.0,4014.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [103]:
# converting the data to bytes array format to make it work with invoke_endpoint method (only for 10 rows)
csv_object = test_data.iloc[:10,1:].to_csv(header=False, index=False).encode("utf-8")
csv_object

b'15,47.0,1.0,1.0,52.0,-1.0,-1.0,3287.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0\n88,50.0,1.0,2.0,62.0,-1.0,-1.0,1807.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0\n0,10.0,1.0,4.0,59.0,76.0,58.0,1519.0,1679.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0\n13,32.0,1.0,2.0,62.0,-1.0,-1.0,1211.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0\n34,47.0,1.0,1.0,59.0,82.0,59.0,4014.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0\n91,3.0,4.0,2.0,62.0,-1.0,-1.0,1807.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0\n60,78.0,1.0,1.0,58.0,-1.0,-1.0,3299.0,4057.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.

In [104]:
# Bulk/Batch prediction
# csv serialization
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=csv_object,
    ContentType="text/csv",
)

output_ = np.array(response["Body"].read().decode('utf-8'))
print(type(output_))

<class 'numpy.ndarray'>


In [96]:
output_ = output_.reshape(-1)[0].split(',')
output_ = [float(i) for i in output_]
output_[:5]

[0.0, 0.0, 1.0, 0.0, 1.0]

In [97]:
actual_output = test_data['target_encoded'].values

from sklearn import metrics

print("Test Data:--------------------------------------")
accuracy = metrics.accuracy_score(actual_output[:10],output_)
print(f"Accuracy: {accuracy}\n")
print(metrics.classification_report(actual_output[:10],output_))

Test Data:--------------------------------------
Accuracy: 0.8

              precision    recall  f1-score   support

           0       0.86      0.86      0.86         7
           1       0.67      0.67      0.67         3

    accuracy                           0.80        10
   macro avg       0.76      0.76      0.76        10
weighted avg       0.80      0.80      0.80        10



### **Invoking the Endpoint: (way-2)**

([Contents:](#Contents:))

In [288]:

# from sagemaker.predictor import Predictor
# import sagemaker
# import time

# predictor = Predictor(endpoint_name=endpoint_name, serializer=sagemaker.serializers.CSVSerializer())
# test_out = predictor.predict(test_data_array).decode('utf-8') # predict!
# print(test_out[:10])

# test_predictions_array = np.fromstring(test_out[:], sep=',') # and turn the prediction into an array
# print(test_predictions_array.shape)


# from sklearn import metrics

# print("Test Data:--------------------------------------")
# accuracy = metrics.accuracy_score(test_data.target_encoded.values,test_predictions_array)
# print(f"Accuracy: {accuracy}\n")
# print(metrics.classification_report(test_data.target_encoded.values,test_predictions_array))

### **Explore the captured data:**

([Contents:](#Contents:))

Now list the data capture files stored in Amazon S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred. The format of the Amazon S3 path is:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [105]:
# Listing the files in the data capture path using boto3
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
model_monitor/datacapture/tuned-xgboost-3/AllTraffic/2022/08/25/06/12-59-860-35037ddb-283e-46ec-909f-c4aac5f8f6a9.jsonl
 model_monitor/datacapture/tuned-xgboost-3/AllTraffic/2022/08/25/06/20-08-455-de16662c-0886-4ee0-b428-1c15e15b2059.jsonl


Next, view the contents of a single capture file. Here you should see all the data captured in an Amazon SageMaker specific JSON-line formatted file. Take a quick peek at the first few lines in the captured file.

In [106]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file)

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"15,47.0,1.0,1.0,52.0,-1.0,-1.0,3287.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0\n88,50.0,1.0,2.0,62.0,-1.0,-1.0,1807.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0\n0,10.0,1.0,4.0,59.0,76.0,58.0,1519.0,1679.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0\n13,32.0,1.0,2.0,62.0,-1.0,-1.0,1211.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0\n34,47.0,1.0,1.0,59.0,82.0,59.0,4014.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0\n91,3.0,4.0,2.0,62.0,-1.0,-1.0,1807.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0\n60,7

**NOTE:** If you send one row at a time to the endpoint for prediction then the json structure you see above will have INPUT and OUTPUT details for single row only

In [107]:
# Beautification of the json using json library
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "15,47.0,1.0,1.0,52.0,-1.0,-1.0,3287.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0\n88,50.0,1.0,2.0,62.0,-1.0,-1.0,1807.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0\n0,10.0,1.0,4.0,59.0,76.0,58.0,1519.0,1679.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0\n13,32.0,1.0,2.0,62.0,-1.0,-1.0,1211.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0\n34,47.0,1.0,1.0,59.0,82.0,59.0,4014.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0\n91,3.0,4.0,2.0,62.0,-1.0,-1.0,1807.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0

<!-- # **Model Data Monitor : Baselining and continuous monitoring:** -->

### **2. Constraint suggestion with baseline/training dataset**

([Contents:](#Contents:))

In addition to collecting the data, Amazon SageMaker provides the capability for you to monitor and evaluate the data observed by the endpoints. For this:
1. Create a baseline with which you compare the realtime traffic. 
1. Once a baseline is ready, setup a schedule to continously evaluate and compare against the baseline.

The training dataset with which you trained the model is usually a good baseline dataset. Note that the training dataset data schema and the inference dataset schema should exactly match (i.e. the number and order of the features).

From the training dataset you can ask Amazon SageMaker to suggest a set of baseline `constraints` and generate descriptive `statistics` to explore the data. For this example, upload the training dataset that was used to train the pre-trained model included in this example. If you already have it in Amazon S3, you can directly point to it.

In [302]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
baseline_prefix = 'model_monitor/baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = 's3://{}/{}'.format(bucket,baseline_data_prefix)
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_results_prefix)
print('Baseline data uri: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))

Baseline data uri: s3://accidentbucket/model_monitor/baselining/data
Baseline results uri: s3://accidentbucket/model_monitor/baselining/results


In [314]:
# uploading the train data to s3 bucket
import os

train_data.drop('target_encoded',axis=1).to_csv("train_data_with_header.csv")

training_data_file = open("train_data_with_header.csv", 'rb')
s3_key = os.path.join(baseline_prefix, 'data', 'training-dataset-with-header.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(training_data_file)

#### Create a baselining job with training dataset

Now that you have the training data ready in Amazon S3, start a job to `suggest` constraints. `DefaultModelMonitor.suggest_baseline(..)` starts a `ProcessingJob` using an Amazon SageMaker provided Model Monitor container to generate the constraints.

In [315]:
# Creating model monitoring job
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=5,
    max_runtime_in_seconds=3600,
)


my_default_monitor_baseline = my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri+'/training-dataset-with-header.csv',
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)


Job Name:  baseline-suggestion-job-2022-08-20-11-52-18-726
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://accidentbucket/model_monitor/baselining/data/training-dataset-with-header.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://accidentbucket/model_monitor/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
.........................2022-08-20 11:56:19,048 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...
2022-08-20 11:56:19.585712: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file

#### Explore the generated constraints and statistics

In [316]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Files:")
print("\n ".join(report_files))

Found Files:
model_monitor/baselining/results/constraints.json
 model_monitor/baselining/results/statistics.json


In [426]:
# Baseline Statistics
import pandas as pd
pd.set_option('display.max_colwidth',50)

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df

,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,_c0,Integral,33184,0,23791.985626,789513251.0,13681.213108,1.0,47405.0,"[{'lower_bound': 1.0, 'upper_bound': 4741.4, '...",0.64,2048.0,"[[], [], [14.0, 34.0, 86.0, 159.0, 216.0, 236...."
1,age_years,Integral,33184,0,30.991532,1028423.0,31.352060,0.0,112.0,"[{'lower_bound': 0.0, 'upper_bound': 11.2, 'co...",0.64,2048.0,"[[], [], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
2,insurance_grp,Fractional,33184,0,42.474958,1409489.0,28.883565,2.0,101.0,"[{'lower_bound': 2.0, 'upper_bound': 11.9, 'co...",0.64,2048.0,"[[], [], [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2..."
3,no_fam_member,Fractional,33184,0,1.466671,48670.0,1.165530,1.0,8.0,"[{'lower_bound': 1.0, 'upper_bound': 1.7, 'cou...",0.64,2048.0,"[[], [], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1..."
4,total_visit_count,Fractional,33184,0,0.917551,30448.0,0.918582,0.0,6.0,"[{'lower_bound': 0.0, 'upper_bound': 0.6, 'cou...",0.64,2048.0,"[[], [], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
5,diagnosis_code_1,Fractional,33184,0,58.958625,1956483.0,5.782184,41.0,74.0,"[{'lower_bound': 41.0, 'upper_bound': 44.3, 'c...",0.64,2048.0,"[[], [], [41.0, 41.0, 41.0, 41.0, 41.0, 41.0, ..."
6,diagnosis_code_2,Fractional,33184,0,23.107883,766812.0,35.311406,-1.0,94.0,"[{'lower_bound': -1.0, 'upper_bound': 8.5, 'co...",0.64,2048.0,"[[], [], [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, ..."
7,diagnosis_code_3,Fractional,33184,0,19.695727,653583.0,29.023339,-1.0,74.0,"[{'lower_bound': -1.0, 'upper_bound': 6.5, 'co...",0.64,2048.0,"[[], [], [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, ..."
8,p1,Fractional,33184,0,2326.318437,77196551.0,1388.762605,106.0,5044.0,"[{'lower_bound': 106.0, 'upper_bound': 599.8, ...",0.64,2048.0,"[[], [], [115.0, 131.0, 214.0, 252.0, 268.0, 2..."
9,p2,Fractional,33184,0,520.735716,17280094.0,1051.882997,0.0,5043.0,"[{'lower_bound': 0.0, 'upper_bound': 504.3, 'c...",0.64,2048.0,"[[], [], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."


In [427]:
# Baseline Constraints
constraints_df = pd.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df

,name,inferred_type,completeness,num_constraints.is_non_negative
0,_c0,Integral,1.0,True
1,age_years,Integral,1.0,True
2,insurance_grp,Fractional,1.0,True
3,no_fam_member,Fractional,1.0,True
4,total_visit_count,Fractional,1.0,True
5,diagnosis_code_1,Fractional,1.0,True
6,diagnosis_code_2,Fractional,1.0,False
7,diagnosis_code_3,Fractional,1.0,False
8,p1,Fractional,1.0,True
9,p2,Fractional,1.0,True


### **3. Creating a Schedule and Analyzing collected data for data quality issues**

([Contents:](#Contents:))

When you have collected the data above, analyze and monitor the data with Monitoring Schedules

#### **Create a schedule**

In [319]:
# # First, copy over some test scripts to the S3 bucket so that they can be used for pre and post processing
# boto3.Session().resource('s3').Bucket(bucket).Object(code_prefix+"/preprocessor.py").upload_file('preprocessor.py')
# boto3.Session().resource('s3').Bucket(bucket).Object(code_prefix+"/postprocessor.py").upload_file('postprocessor.py')

You can create a model monitoring schedule for the endpoint created earlier. Use the baseline resources (constraints and statistics) to compare against the realtime traffic.

In [362]:
# creating schedule using cron
from sagemaker.model_monitor import CronExpressionGenerator 
from time import gmtime, strftime

mon_schedule_name = 'DEMO-Death-Classification-Model' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
s3_report_path = 's3://{}/{}'.format(bucket,'report')

my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint_name,
    #record_preprocessor_script=pre_processor_script,
#     post_analytics_processor_script=s3_code_postprocessor_uri,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,

)

#### **Generating some artificial traffic**

([Contents:](#Contents:))

The cell below starts a thread to send some traffic to the endpoint. Note that you need to stop the kernel to terminate this thread. If there is no traffic, the monitoring jobs are marked as `Failed` since there is no data to process.

In [437]:
predictor.endpoint_name

'tuned-xgboost-2'

In [8]:
import random
import pandas as pd

test_samples = []

for i in range(10):
    test_samples.append([random.randint(1,256) for _ in range(36)])

print(len(test_samples))
print(len(test_samples[0]))
print(test_samples)

test_df = pd.DataFrame(test_samples)
test_df.to_csv('test_artificial_data.csv',header=False, index=False)

10
36
[[208, 246, 203, 137, 244, 181, 70, 256, 144, 117, 242, 64, 104, 220, 137, 190, 252, 88, 41, 175, 203, 182, 138, 6, 107, 51, 133, 44, 179, 74, 140, 247, 57, 128, 53, 164], [127, 238, 245, 195, 71, 149, 123, 29, 195, 122, 199, 215, 158, 239, 198, 167, 36, 230, 3, 112, 245, 78, 247, 22, 65, 143, 19, 231, 62, 171, 15, 171, 80, 124, 99, 247], [101, 235, 176, 128, 213, 206, 135, 173, 73, 70, 52, 36, 14, 134, 55, 52, 156, 58, 174, 133, 51, 256, 92, 149, 244, 5, 43, 8, 68, 35, 31, 187, 149, 98, 132, 178], [126, 52, 48, 119, 189, 65, 255, 126, 20, 219, 51, 158, 14, 188, 104, 100, 110, 246, 170, 52, 253, 216, 119, 70, 175, 74, 193, 132, 90, 195, 193, 33, 234, 175, 87, 231], [110, 100, 195, 206, 167, 139, 115, 24, 158, 210, 46, 9, 130, 60, 238, 192, 237, 34, 137, 30, 76, 13, 60, 113, 245, 101, 214, 190, 2, 47, 5, 244, 184, 236, 130, 142], [224, 98, 168, 72, 4, 157, 1, 49, 92, 148, 1, 32, 40, 113, 251, 128, 166, 49, 106, 28, 87, 72, 44, 197, 1, 188, 171, 126, 205, 229, 12, 47, 9, 121, 175, 

In [440]:
# Method of passing artificial data
from threading import Thread
from time import sleep
import time

endpoint_name=predictor.endpoint_name
runtime_client = boto3.client('runtime.sagemaker')

# (just repeating code from above for convenience/ able to run this section independently)
def invoke_endpoint(ep_name, file_name, runtime_client):
    with open(file_name, 'r') as f:
        for row in f:
            payload = row.rstrip('\n')
            print(payload)
            response = runtime_client.invoke_endpoint(EndpointName=ep_name,
                                          ContentType='text/csv', 
                                          Body=payload)
            response['Body'].read()
            time.sleep(1)
            
def invoke_endpoint_forever():
    while True:
        invoke_endpoint(endpoint_name, 'test_artificial_data.csv', runtime_client)
        
thread = Thread(target = invoke_endpoint_forever)
thread.start()

# Note that you need to stop the kernel to stop the invocations

28,173,127,105,13,116,186,12,179,157,158,79,25,37,214,205,73,151,148,152,139,6,83,108,35,135,169,108,168,52,81,29,94,245,61,44
123,28,9,152,157,251,123,255,80,185,139,244,230,80,84,238,71,107,218,213,4,255,175,88,87,105,106,188,105,204,47,177,105,236,233,240
207,216,252,126,170,98,33,132,72,101,34,201,203,75,214,228,230,177,167,170,239,72,39,172,111,117,199,195,233,65,206,185,73,54,85,29
240,236,65,60,126,6,226,238,97,151,184,126,249,238,42,155,19,227,227,239,196,8,94,83,16,225,88,61,148,58,162,45,70,32,65,161
39,110,65,101,131,99,230,66,88,102,248,227,24,27,224,121,242,214,32,231,202,158,23,201,144,101,167,133,62,187,75,40,173,48,59,131
158,75,171,192,228,232,94,115,94,64,204,108,78,93,132,203,40,92,39,10,25,44,251,180,73,116,38,220,71,97,108,158,246,205,100,91
28,173,127,105,13,116,186,12,179,157,158,79,25,37,214,205,73,151,148,152,139,6,83,108,35,135,169,108,168,52,81,29,94,245,61,44
10,110,90,39,75,235,164,213,120,109,134,110,52,216,38,96,68,137,60,98,110,43,193,136,32,251,92,217,1

#### **Describe and inspect the schedule**

([Contents:](#Contents:))

Once you describe, observe that the MonitoringScheduleStatus changes to Scheduled.

In [3]:
# desc_schedule_result = my_default_monitor.describe_schedule()
# print('Schedule status: {}'.format(desc_schedule_result['MonitoringScheduleStatus']))

### **Analyze the Executions**

([Contents:](#Contents:))

The schedule starts jobs at the previously specified intervals. Here, you list the latest five executions. Note that if you are kicking this off after creating the hourly schedule, you might find the executions empty. You might have to wait until you cross the hour boundary (in UTC) to see executions kick off. The code below has the logic for waiting.

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule your execution. You might see your execution start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.

In [412]:
mon_executions = my_default_monitor.list_executions()
print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_default_monitor.list_executions()    

We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.
We will have to wait till we hit the hour...


### Inspect a specific execution (latest execution)
In the previous cell, you picked up the latest completed or failed scheduled execution. Here are the possible terminal states and what each of them mean: 
* Completed - This means the monitoring execution completed and no issues were found in the violations report.
* CompletedWithViolations - This means the execution completed, but constraint violations were detected.
* Failed - The monitoring execution failed, maybe due to client error (perhaps incorrect role premissions) or infrastructure issues. Further examination of FailureReason and ExitMessage is necessary to identify what exactly happened.
* Stopped - job exceeded max runtime or was manually stopped.

In [413]:
latest_execution = mon_executions[-1] # latest execution's index is -1, second to last is -2 and so on..
# time.sleep(60)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

!Latest execution status: Completed
Latest execution result: CompletedWithViolations: Job completed successfully with 1 violations.


In [414]:
report_uri=latest_execution.output.destination
print('Report Uri: {}'.format(report_uri))

Report Uri: s3://accidentbucket/report/tuned-xgboost-2/DEMO-Death-Classification-Model2022-08-20-14-23-11/2022/08/20/16


### List the generated reports

In [6]:
# cell 23
import boto3
from urllib.parse import urlparse

report_uri = 's3://accidentbucket/report/tuned-xgboost-2/DEMO-Death-Classification-Model2022-08-20-14-23-11/2022/08/20/16'
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print('Report bucket: {}'.format(report_bucket))
print('Report key: {}'.format(report_key))

s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))

Report bucket: accidentbucket
Report key: report/tuned-xgboost-2/DEMO-Death-Classification-Model2022-08-20-14-23-11/2022/08/20/16
Found Report Files:
report/tuned-xgboost-2/DEMO-Death-Classification-Model2022-08-20-14-23-11/2022/08/20/16/constraint_violations.json


### Violations report

If there are any violations compared to the baseline, they will be listed here.

In [416]:
# cell 24
violations = my_default_monitor.latest_monitoring_constraint_violations()
pd.set_option('display.max_colwidth', None)
constraints_df = pd.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

,feature_name,constraint_check_type,description
0,Extra columns,extra_column_check,"There are extra columns in current dataset. Number of columns in current dataset: 86, Number of columns in baseline constraints: 37"


### How to check the status and executions of the scheduled Data Monitor: (OPTIONAL)

In [18]:
from sagemaker.model_monitor import DefaultModelMonitor

In [32]:
# s3://accidentbucket/report/tuned-xgboost-2/DEMO-Death-Classification-Model2022-08-20-14-23-11/2022/08/
mon_schedule_name_tuned_2 = 'DEMO-Death-Classification-Model2022-08-20-14-23-11'

In [37]:
import sagemaker
my_monitor = DefaultModelMonitor.attach(monitor_schedule_name=mon_schedule_name_tuned_2,sagemaker_session=sagemaker.session.Session())

In [38]:
mon_executions = my_monitor.list_executions()
print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_monitor.list_executions()    

We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.
We will have to wait till we hit the hour...


In [39]:
latest_execution = mon_executions[-1] # latest execution's index is -1, second to last is -2 and so on..
# time.sleep(60)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

!Latest execution status: Completed
Latest execution result: CompletedWithViolations: Job completed successfully with 36 violations.


In [40]:
desc_schedule_result = my_monitor.describe_schedule()
print('Schedule status: {}'.format(desc_schedule_result['MonitoringScheduleStatus']))

Schedule status: Scheduled


In [41]:
my_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:143176219551:monitoring-schedule/demo-death-classification-model2022-08-20-14-23-11',
 'MonitoringScheduleName': 'DEMO-Death-Classification-Model2022-08-20-14-23-11',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2022, 8, 20, 14, 23, 12, 347000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 8, 26, 9, 17, 9, 740000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'data-quality-job-definition-2022-08-20-14-23-12-083',
  'MonitoringType': 'DataQuality'},
 'EndpointName': 'tuned-xgboost-2',
 'LastMonitoringExecutionSummary': {'MonitoringScheduleName': 'DEMO-Death-Classification-Model2022-08-20-14-23-11',
  'ScheduledTime': datetime.datetime(2022, 8, 26, 9, 0, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2022, 8, 26, 9, 9, 32, 148000, tzinfo=tzlocal(

In [42]:
# cell 24
violations = my_monitor.latest_monitoring_constraint_violations()
pd.set_option('display.max_colwidth', None)
constraints_df = pd.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

,feature_name,constraint_check_type,description
0,country_of_origin_M,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
1,sex_M,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
2,place_of_incident_Public property,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
3,country_of_origin_L,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
4,insurance_grp,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
5,diagnosis_code_1,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
6,place_of_death_1,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
7,diagnosis_code_3,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
8,_c0,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
9,place_of_incident_Unknown,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."


In [44]:
latest_execution = mon_executions[-1] # latest execution's index is -1, second to last is -2 and so on..
# time.sleep(60)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

!Latest execution status: Completed
Latest execution result: CompletedWithViolations: Job completed successfully with 36 violations.


In [46]:
mon_executions = my_monitor.list_executions()
len(mon_executions)

1

In [47]:
constraints_df

,feature_name,constraint_check_type,description
0,country_of_origin_M,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
1,sex_M,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
2,place_of_incident_Public property,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
3,country_of_origin_L,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
4,insurance_grp,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
5,diagnosis_code_1,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
6,place_of_death_1,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
7,diagnosis_code_3,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
8,_c0,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
9,place_of_incident_Unknown,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
